# Load data

In [1]:
import pandas as pd

/tmp/ipykernel_5829/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from pathlib import Path

def load_data(path: str) -> pd.DataFrame:
    data = pd.read_csv(Path(path))
    return data

In [3]:
data = load_data("../data/data.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


# Preprocessing

In [5]:
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
from sentence_transformers import SentenceTransformer

# intfloat/e5-small-v2 or intfloat/e5-base-v2 or intfloat/e5-large-v2
model = SentenceTransformer("intfloat/e5-large-v2")

In [7]:
import torch

if torch.cuda.is_available():
    print("GPU is available and accessible.")
else:
    print("GPU is not available or accessible.")


GPU is available and accessible.


In [13]:
embeddings = model.encode(data['text'], normalize_embeddings=True, show_progress_bar=True, device="cuda:0")

Batches:   0%|          | 0/238 [00:00<?, ?it/s]

In [9]:
nb_comp = len(embeddings[0])
nb_data = len(embeddings)
print(f"embedding size : {nb_comp}")

embedding size : 1024


In [10]:
for num_comp in range(nb_comp):
    data['x_' + str(num_comp)] = [embeddings[k][num_comp] for k in range(nb_data)]

In [11]:
data

,id,keyword,location,text,target,x_0,x_1,x_2,x_3,x_4,...,x_1014,x_1015,x_1016,x_1017,x_1018,x_1019,x_1020,x_1021,x_1022,x_1023
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.011803,-0.031025,0.034121,-0.020263,-0.009868,...,0.003252,-0.009478,0.014789,0.010092,0.004460,-0.016296,0.069081,-0.030228,0.015628,0.037624
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.011739,-0.013322,0.018322,0.001877,-0.020319,...,0.029806,-0.005918,0.028307,0.035091,-0.008006,-0.024362,0.050332,-0.052541,0.033290,0.006768
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.016125,-0.040696,0.013457,-0.034184,-0.030858,...,0.017645,-0.018340,0.033430,0.038974,0.044836,-0.028837,0.032119,-0.033722,0.016990,0.014847
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.027257,-0.038108,0.032964,-0.001223,-0.010603,...,0.035597,-0.016341,0.016920,0.024618,0.036177,-0.036252,0.023896,-0.023695,0.003887,0.027429
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.024751,-0.048040,0.021240,-0.013732,-0.037089,...,0.030447,-0.007058,0.007583,0.054259,0.026876,-0.016711,0.023218,-0.028445,0.035993,-0.002563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,0.009944,-0.023126,0.010891,-0.024258,-0.054451,...,-0.005942,-0.006825,0.000577,-0.002899,0.026869,-0.005748,0.047420,-0.009907,0.039656,0.029818
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,0.027626,-0.040357,0.042238,-0.002208,-0.007951,...,0.011700,-0.023696,0.031429,0.001988,0.039541,-0.018200,0.052215,-0.027219,0.005467,-0.016417
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,0.005850,-0.071938,0.029449,-0.000528,-0.048940,...,0.027868,-0.013050,0.006874,0.003968,0.025952,-0.007909,0.032589,-0.009245,0.023993,0.013514
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,0.012210,-0.041141,0.012503,-0.010618,-0.019101,...,-0.004625,-0.022072,0.033457,0.007907,0.016308,-0.011299,0.053951,-0.036444,0.021340,0.015396


# Training

## K-NN

## Linear regression

## Ridge regression

## Lasso regression

## BERT

# Evaluation